In [10]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil
import pandas as pd
import BacktestingStrategies.Optimization as opt

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('1655.T', 'yahoo', start, end)   #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)    #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)    #MAXIS ナスダック100上場投信
# data = web.DataReader('2559.T', 'yahoo', start, end)      #MAXIS 全世界株式(オール・カントリー)上場投信
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

              High     Low    Open   Close       Volume  Adj Close
Date                                                              
2017-10-06  1582.0  1574.0  1576.0  1577.0    8602389.0     1577.0
2017-10-09  1577.0  1577.0  1577.0  1577.0          0.0     1577.0
2017-10-10  1583.0  1557.0  1581.0  1558.0   12064480.0     1558.0
2017-10-11  1562.0  1545.0  1561.0  1549.0    9648040.0     1549.0
2017-10-12  1543.0  1533.0  1536.0  1538.0   12519070.0     1538.0
...            ...     ...     ...     ...          ...        ...
2022-09-29   400.0   391.0   392.0   393.0   88373873.0      393.0
2022-09-30   413.0   398.0   400.0   410.0  115732903.0      410.0
2022-10-03   419.0   399.0   415.0   400.0  131503576.0      400.0
2022-10-04   387.0   375.0   386.0   375.0  114901364.0      375.0
2022-10-06   372.0   366.0   371.0   368.0   29357388.0      368.0

[1235 rows x 6 columns]


In [11]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [12]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**ボリンジャーバンドとRSI組み合わせ**

In [4]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "n": range(10, 40, 5),\
    "nu": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "nd": np.arange(2.0, 3.1, 0.1).round(2).tolist(),\
    "upper_bound": range(50, 85, 5),
    "lower_bound": range(10, 45, 5),
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   47.087379
Equity Final [$]                8214811.80925
Equity Peak [$]                 16236815.4523
Return [%]                         -17.851882
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -3.929984
Volatility (Ann.) [%]               23.016954
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -59.203876
Avg. Drawdown [%]                   -7.715571
Max. Drawdown Duration     1111 days 00:00:00
Avg. Drawdown Duration      105 days 00:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                      11.991766
Worst Trade [%]                    -31.980323
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7780,52,98,1285.33105,1389.0,8.065444e+05,0.080655,2017-12-04,2018-02-06,64 days
1,8961,167,180,1205.94000,1260.0,4.844317e+05,0.044828,2018-05-14,2018-05-31,17 days
2,10520,259,284,1073.28660,1191.0,1.238345e+06,0.109676,2018-09-19,2018-10-24,35 days
3,10984,313,318,1140.61825,1267.0,1.388177e+06,0.110801,2018-12-04,2018-12-11,7 days
4,11241,360,387,1238.09840,1212.0,-2.933721e+05,-0.021079,2019-02-14,2019-03-26,40 days
5,12506,402,416,1089.36580,1220.0,1.633711e+06,0.119918,2019-04-16,2019-05-14,28 days
6,13690,451,474,1114.48955,1186.0,9.789781e+05,0.064164,2019-07-02,2019-08-05,34 days
7,14147,497,592,1147.65290,910.0,-3.362076e+06,-0.207077,2019-09-06,2020-01-31,147 days
8,14171,668,714,908.47480,814.0,-1.338802e+06,-0.103993,2020-05-27,2020-08-03,68 days
9,15965,722,775,722.55905,714.0,-1.366452e+05,-0.011845,2020-08-14,2020-11-02,80 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   28.398058
Equity Final [$]                  8236652.607
Equity Peak [$]                11932327.17925
Return [%]                         -17.633474
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -3.877962
Volatility (Ann.) [%]               15.682532
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -39.214141
Avg. Drawdown [%]                   -8.321347
Max. Drawdown Duration     1160 days 00:00:00
Avg. Drawdown Duration      179 days 00:00:00
# Trades                                   22
Win Rate [%]                             50.0
Best Trade [%]                       7.540591
Worst Trade [%]                      -18.9878
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7184,27,40,1391.85575,1386.0,-4.206771e+04,-0.004207,2017-10-30,2017-11-16,17 days
1,8361,76,94,1190.86575,1227.0,3.021185e+05,0.030343,2018-01-05,2018-01-31,26 days
2,9559,259,274,1073.28660,1072.0,-1.229861e+04,-0.001199,2018-09-19,2018-10-10,21 days
3,8984,313,315,1140.61825,1213.0,6.502776e+05,0.063458,2018-12-04,2018-12-06,2 days
4,8802,360,377,1238.09840,1231.0,-6.248012e+04,-0.005733,2019-02-14,2019-03-11,25 days
5,9946,402,412,1089.36580,1134.0,4.439318e+05,0.040973,2019-04-16,2019-05-08,22 days
6,10120,451,463,1114.48955,1163.0,4.909258e+05,0.043527,2019-07-02,2019-07-19,17 days
7,10256,497,515,1147.65290,1104.0,-4.477041e+05,-0.038037,2019-09-06,2019-10-04,28 days
8,11334,523,556,998.92030,914.0,-9.624867e+05,-0.085012,2019-10-17,2019-12-05,49 days
9,11918,563,575,869.28175,906.0,4.376081e+05,0.042240,2019-12-16,2020-01-07,22 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7780,52,98,1285.33105,1389.0,8.065444e+05,0.080655,2017-12-04,2018-02-06,64 days
1,-8283,99,100,1304.51055,1382.0,-6.418451e+05,-0.059401,2018-02-07,2018-02-08,1 days
2,-7020,101,102,1447.79775,1422.0,1.811002e+05,0.017819,2018-02-09,2018-02-12,3 days
3,-7507,103,104,1378.14425,1439.0,-4.568441e+05,-0.044158,2018-02-13,2018-02-14,1 days
4,-6482,132,180,1525.41165,1260.0,1.720398e+06,0.173993,2018-03-26,2018-05-31,66 days
5,-9107,203,204,1274.65905,1311.0,-3.309570e+05,-0.028510,2018-07-03,2018-07-04,1 days
6,-8678,205,206,1299.53530,1308.0,-7.345667e+04,-0.006514,2018-07-05,2018-07-06,1 days
7,-8994,233,284,1245.80260,1191.0,4.928946e+05,0.043990,2018-08-14,2018-10-24,71 days
8,-9285,286,287,1259.73330,1283.0,-2.160313e+05,-0.018470,2018-10-26,2018-10-29,3 days
9,-8721,288,318,1316.45115,1267.0,4.312635e+05,0.037564,2018-10-30,2018-12-11,42 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   68.446602
Equity Final [$]                 13271925.331
Equity Peak [$]                 17507175.3626
Return [%]                          32.719253
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                    5.941032
Volatility (Ann.) [%]               31.217804
Sharpe Ratio                         0.190309
Sortino Ratio                        0.280002
Calmar Ratio                         0.104808
Max. Drawdown [%]                  -56.684828
Avg. Drawdown [%]                   -6.356178
Max. Drawdown Duration      533 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                   86
Win Rate [%]                        51.162791
Best Trade [%]                      38.190282
Worst Trade [%]                     -21.10444
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7184,27,40,1391.85575,1386.0,-42067.70800,-0.004207,2017-10-30,2017-11-16,17 days
1,-7479,55,62,1331.37690,1285.0,346852.83510,0.034834,2017-12-07,2017-12-18,11 days
2,8653,76,94,1190.86575,1227.0,312669.66525,0.030343,2018-01-05,2018-01-31,26 days
3,-7681,98,99,1382.12445,1311.0,546306.90045,0.051460,2018-02-06,2018-02-07,1 days
4,-8118,100,101,1375.15910,1455.0,-648148.42620,-0.058059,2018-02-08,2018-02-09,1 days
...,...,...,...,...,...,...,...,...,...,...
81,-29190,1146,1151,433.84180,418.0,462422.14200,0.036515,2022-05-13,2022-05-20,7 days
82,34014,1162,1168,385.90080,415.0,989780.18880,0.075406,2022-06-06,2022-06-14,8 days
83,-34349,1181,1189,410.95565,412.0,-35872.37815,-0.002541,2022-07-01,2022-07-13,12 days
84,37362,1194,1221,376.85625,358.0,-704507.21250,-0.050036,2022-07-21,2022-08-30,40 days


<Strategy BBandRSI_WithShortPosition(upper_bound=50,lower_bound=10,rsi_window=10)>

**ボリンジャーバンドとRSI組み合わせ EntryRSI, ExitBolinger**

In [5]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.EntryRSI50andExitBB, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.EntryRSI50andExitBB_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   32.443366
Equity Final [$]                 7332736.7276
Equity Peak [$]                  12690963.803
Return [%]                         -26.672633
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -6.129317
Volatility (Ann.) [%]               21.231635
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -51.163213
Avg. Drawdown [%]                  -14.860971
Max. Drawdown Duration     1541 days 00:00:00
Avg. Drawdown Duration      332 days 00:00:00
# Trades                                   46
Win Rate [%]                        47.826087
Best Trade [%]                      13.332023
Worst Trade [%]                    -12.308543
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7437,55,57,1344.62310,1272.0,-5.400980e+05,-0.054010,2017-12-07,2017-12-11,4 days
1,7671,94,98,1233.07365,1389.0,1.196111e+06,0.126453,2018-01-31,2018-02-06,6 days
2,7877,115,132,1352.66270,1533.0,1.420517e+06,0.133320,2018-03-01,2018-03-26,25 days
3,9683,176,180,1247.14295,1260.0,1.244948e+05,0.010309,2018-05-25,2018-05-31,6 days
4,9822,194,203,1242.11820,1281.0,3.818970e+05,0.031303,2018-06-20,2018-07-03,13 days
5,9921,210,214,1268.24690,1165.0,-1.024312e+06,-0.081409,2018-07-12,2018-07-18,6 days
6,9230,232,233,1252.16770,1252.0,-1.547871e+03,-0.000134,2018-08-13,2018-08-14,1 days
7,9488,251,257,1217.99940,1133.0,-8.064743e+05,-0.069786,2018-09-07,2018-09-17,10 days
8,9166,276,284,1172.77665,1191.0,1.670352e+05,0.015539,2018-10-12,2018-10-24,12 days
9,9098,293,313,1199.91030,1135.0,-5.905539e+05,-0.054096,2018-11-06,2018-12-04,28 days


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   35.194175
Equity Final [$]                6026984.15305
Equity Peak [$]                 12087051.0991
Return [%]                         -39.730158
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -9.808413
Volatility (Ann.) [%]               20.161484
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -56.392829
Avg. Drawdown [%]                  -13.963821
Max. Drawdown Duration     1499 days 00:00:00
Avg. Drawdown Duration      287 days 00:00:00
# Trades                                   72
Win Rate [%]                        44.444444
Best Trade [%]                      13.332023
Worst Trade [%]                     -22.26449
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7437,55,57,1344.62310,1272.0,-5.400980e+05,-0.054010,2017-12-07,2017-12-11,4 days
1,7671,94,98,1233.07365,1389.0,1.196111e+06,0.126453,2018-01-31,2018-02-06,6 days
2,-8308,113,114,1282.61945,1303.0,-1.693216e+05,-0.015890,2018-02-27,2018-02-28,1 days
3,7752,115,132,1352.66270,1533.0,1.397975e+06,0.133320,2018-03-01,2018-03-26,25 days
4,-8775,137,138,1354.26305,1408.0,-4.715417e+05,-0.039680,2018-04-02,2018-04-03,1 days
...,...,...,...,...,...,...,...,...,...,...
67,-17304,1148,1149,413.94080,407.0,1.201036e+05,0.016768,2022-05-17,2022-05-18,1 days
68,17337,1151,1162,420.06910,384.0,-6.253300e+05,-0.085865,2022-05-20,2022-06-06,17 days
69,15963,1168,1194,417.05425,375.0,-6.713120e+05,-0.100836,2022-06-14,2022-07-21,37 days
70,16639,1221,1232,359.77210,363.0,5.370903e+04,0.008972,2022-08-30,2022-09-14,15 days


**MACDとRSIの組み合わせ**

In [13]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "MACDThreshold": range(-100, 100, 10),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                    4.696356
Equity Final [$]                 9472772.9854
Equity Peak [$]                 10189730.9854
Return [%]                           -5.27227
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                   -1.099112
Volatility (Ann.) [%]                6.407377
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.914482
Avg. Drawdown [%]                   -6.031329
Max. Drawdown Duration     1050 days 00:00:00
Avg. Drawdown Duration      528 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -5.272395
Worst Trade [%]                     -5.272395
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,10863,530,587,920.5342,872.0,-527227.0146,-0.052724,2019-11-14,2020-02-10,88 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 498 configurations.
  output = _optimize_grid()


Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                    42.42915
Equity Final [$]                9382455.66895
Equity Peak [$]                 13698479.9854
Return [%]                          -6.175443
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                   -1.292256
Volatility (Ann.) [%]               26.023554
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -38.70718
Avg. Drawdown [%]                  -13.363122
Max. Drawdown Duration      668 days 00:00:00
Avg. Drawdown Duration      195 days 00:00:00
# Trades                                    2
Win Rate [%]                             50.0
Best Trade [%]                      36.985676
Worst Trade [%]                    -31.507867
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,10863,530,606,920.53420,1261.0,3.698480e+06,0.369857,2019-11-14,2020-03-10,117 days
1,25289,788,1234,541.66805,371.0,-4.316024e+06,-0.315079,2020-12-08,2022-10-06,667 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7665,88,89,1304.51055,1382.0,-5.939566e+05,-0.059401,2018-02-07,2018-02-08,1 days
1,-6647,91,92,1414.96110,1385.0,1.991514e+05,0.021175,2018-02-12,2018-02-13,1 days
2,-6708,93,94,1431.87695,1420.0,7.967058e+04,0.008295,2018-02-14,2018-02-15,1 days
3,-7114,99,112,1361.22840,1359.0,1.585284e+04,0.001637,2018-02-22,2018-03-13,19 days
4,-7131,125,200,1360.23335,1214.0,1.042790e+06,0.107506,2018-03-30,2018-07-13,105 days
5,-8528,275,276,1259.73330,1283.0,-1.984184e+05,-0.018470,2018-10-26,2018-10-29,3 days
6,-8010,277,282,1316.45115,1194.0,9.808337e+05,0.093016,2018-10-30,2018-11-06,7 days
7,-9693,297,318,1189.08475,1505.0,-3.062167e+06,-0.265679,2018-11-27,2018-12-26,29 days
8,-6141,319,325,1378.14425,1340.0,2.342438e+05,0.027678,2018-12-27,2019-01-09,13 days
9,-6855,419,420,1268.68875,1264.0,3.214138e+04,0.003696,2019-06-03,2019-06-04,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 555 configurations.
  output = _optimize_grid()


Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   58.866397
Equity Final [$]                 4330669.8506
Equity Peak [$]                   10374258.44
Return [%]                         -56.693301
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                   -15.69767
Volatility (Ann.) [%]               23.622514
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -70.998733
Avg. Drawdown [%]                  -24.694005
Max. Drawdown Duration     1717 days 00:00:00
Avg. Drawdown Duration      577 days 00:00:00
# Trades                                  210
Win Rate [%]                        44.285714
Best Trade [%]                      16.055061
Worst Trade [%]                     -21.10444
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-8480,65,87,1179.13425,1389.0,-1.779662e+06,-0.177983,2018-01-05,2018-02-06,32 days
1,-6301,88,89,1304.51055,1382.0,-4.882610e+05,-0.059401,2018-02-07,2018-02-08,1 days
2,-5340,90,91,1447.79775,1422.0,1.377600e+05,0.017819,2018-02-09,2018-02-12,3 days
3,-5710,92,93,1378.14425,1439.0,-3.474863e+05,-0.044158,2018-02-13,2018-02-14,1 days
4,-5323,94,95,1412.97100,1395.0,9.565963e+04,0.012719,2018-02-15,2018-02-16,1 days
...,...,...,...,...,...,...,...,...,...,...
205,-12966,1224,1226,360.20810,383.0,-2.955198e+05,-0.063274,2022-09-20,2022-09-22,2 days
206,-11188,1227,1228,391.05465,396.0,-5.532858e+04,-0.012646,2022-09-26,2022-09-27,1 days
207,-10880,1229,1230,397.02495,392.0,5.467146e+04,0.012657,2022-09-28,2022-09-29,1 days
208,-10990,1231,1232,398.02000,415.0,-1.866102e+05,-0.042661,2022-09-30,2022-10-03,3 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10,MACDThreshold=-100,upper_bound=50,lower_bound=35,rsi_window=22)>

**MACDとボリンジャーバンドの組み合わせ**

In [14]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "MACDshort": range(10, 25, 5),\
    "MACDlong": range(10, 75, 10),\
    "MACDsignal": range(10, 15, 5),\
    "MACDThreshold": range(-100, 100, 10),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   13.198381
Equity Final [$]                7659421.02015
Equity Peak [$]                    10000000.0
Return [%]                          -23.40579
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                   -5.295559
Volatility (Ann.) [%]               10.863258
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                    -34.5054
Avg. Drawdown [%]                    -34.5054
Max. Drawdown Duration     1658 days 00:00:00
Avg. Drawdown Duration     1658 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                     -23.405886
Worst Trade [%]                    -23.405886
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6491,121,283,1540.58835,1180.0,-2.340579e+06,-0.234059,2018-03-26,2018-11-07,226 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=20,MACDsignal=10,MACDThreshold=10)>

Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   33.765182
Equity Final [$]                7404409.37895
Equity Peak [$]                13239963.87865
Return [%]                         -25.955906
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                   -5.947638
Volatility (Ann.) [%]              160.263154
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -93.293315
Avg. Drawdown [%]                   -7.624358
Max. Drawdown Duration     1260 days 00:00:00
Avg. Drawdown Duration       88 days 00:00:00
# Trades                                   11
Win Rate [%]                        63.636364
Best Trade [%]                      11.202242
Worst Trade [%]                    -42.793986
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7346,99,112,1361.22840,1359.0,1.636983e+04,0.001637,2018-02-22,2018-03-13,19 days
1,-7363,125,200,1360.23335,1214.0,1.076716e+06,0.107506,2018-03-30,2018-07-13,105 days
2,-9336,282,297,1188.08970,1195.0,-6.451456e+04,-0.005816,2018-11-06,2018-11-27,21 days
3,-8271,325,425,1333.36700,1184.0,1.235414e+06,0.112022,2019-01-09,2019-06-11,153 days
4,-9766,477,480,1255.75310,1226.0,2.905688e+05,0.023693,2019-08-26,2019-08-29,3 days
5,-14469,587,617,867.68360,1239.0,-5.372577e+06,-0.427940,2020-02-10,2020-03-26,45 days
6,-16707,900,950,429.86160,444.0,-2.362102e+05,-0.032891,2021-05-26,2021-08-06,72 days
7,-15864,962,999,437.82200,389.0,7.745122e+05,0.111511,2021-08-25,2021-10-19,55 days
8,-19543,1037,1073,395.03485,422.0,-5.269799e+05,-0.068260,2021-12-14,2022-02-07,55 days
9,-16580,1099,1139,433.84180,428.0,9.685704e+04,0.013465,2022-03-17,2022-05-19,63 days


Start                     2017-10-06 00:00:00
End                       2022-10-06 00:00:00
Duration                   1826 days 00:00:00
Exposure Time [%]                   53.927126
Equity Final [$]                14021482.7947
Equity Peak [$]                 15332584.7947
Return [%]                          40.214828
Buy & Hold Return [%]              -76.664553
Return (Ann.) [%]                    7.140359
Volatility (Ann.) [%]               32.825874
Sharpe Ratio                         0.217522
Sortino Ratio                        0.344473
Calmar Ratio                         0.157244
Max. Drawdown [%]                  -45.409321
Avg. Drawdown [%]                    -6.53997
Max. Drawdown Duration      867 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                   25
Win Rate [%]                             48.0
Best Trade [%]                       27.13548
Worst Trade [%]                    -16.017324
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-8480,65,99,1179.13425,1368.0,-1.601582e+06,-0.160173,2018-01-05,2018-02-22,48 days
1,-6317,111,125,1329.38680,1367.0,-2.376026e+05,-0.028294,2018-03-12,2018-03-30,18 days
2,-6909,157,162,1181.12435,1174.0,4.922213e+04,0.006032,2018-05-15,2018-05-22,7 days
3,-7039,177,200,1166.19860,1214.0,-3.364741e+05,-0.040989,2018-06-12,2018-07-13,31 days
4,-6615,219,231,1190.07980,1181.0,6.006288e+04,0.007630,2018-08-09,2018-08-27,18 days
5,-6969,245,283,1138.33720,1180.0,-2.903481e+05,-0.036600,2018-09-14,2018-11-07,54 days
6,-6244,296,325,1223.91150,1340.0,-7.248566e+05,-0.094850,2018-11-26,2019-01-09,44 days
7,-6072,382,425,1139.33225,1184.0,-2.712226e+05,-0.039205,2019-04-03,2019-06-11,69 days
8,-6040,457,460,1100.52530,1120.0,-1.176272e+05,-0.017696,2019-07-26,2019-07-31,5 days
9,-5199,477,480,1255.75310,1226.0,1.546864e+05,0.023693,2019-08-26,2019-08-29,3 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10,MACDThreshold=-100)>

SMA and RSI

In [8]:
import BacktestingStrategies.Strategy_SMAandRSI as btstSMAandRsi
importlib.reload(btstSMAandRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMAandRsi.EntryRSIandExitSMA_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)


# #最適化
opt_params = {
    "SMA_short": range(10, 25, 5),\
    "SMA_long": range(10, 75, 10),\
    "upper_bound": range(50, 85, 5),\
    "lower_bound": range(10, 45, 5),\
    "rsi_window": range(10, 30, 2),\
    "maximize": opt.optim_func,\
    "method": 'grid',\
    "constraint": lambda x: (x.SMA_short < x.SMA_long),\
    "max_tries": 1000}
execute_optimize(bt, opt_params)

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMAandRsi.SMAandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )
    execute_backtest(bt)

    # #最適化
    execute_optimize(bt, opt_params)

Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   39.967638
Equity Final [$]                10508894.8168
Equity Peak [$]                 15415839.9964
Return [%]                           5.088948
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                    1.017153
Volatility (Ann.) [%]               42.682151
Sharpe Ratio                         0.023831
Sortino Ratio                        0.044219
Calmar Ratio                         0.015574
Max. Drawdown [%]                  -65.309396
Avg. Drawdown [%]                   -7.995797
Max. Drawdown Duration     1083 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                   30
Win Rate [%]                        33.333333
Best Trade [%]                      20.607183
Worst Trade [%]                    -11.081161
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7515,31,35,1330.55380,1257.80,-5.467448e+05,-0.054679,2017-11-03,2017-11-09,6 days
1,7169,36,75,1318.49440,1217.90,-7.211613e+05,-0.076295,2017-11-10,2018-01-04,55 days
2,7332,76,85,1190.86575,1136.00,-4.022757e+05,-0.046072,2018-01-05,2018-01-18,13 days
3,7232,89,98,1151.67270,1389.00,1.716351e+06,0.206072,2018-01-24,2018-02-06,13 days
4,-7305,100,128,1375.15910,1417.00,-3.056478e+05,-0.030426,2018-02-08,2018-03-20,40 days
5,8077,167,182,1205.94000,1238.00,2.589486e+05,0.026585,2018-05-14,2018-06-04,21 days
6,9316,259,264,1073.28660,1045.00,-2.635180e+05,-0.026355,2018-09-19,2018-09-26,7 days
7,9324,265,277,1044.14305,1185.65,1.319411e+06,0.135524,2018-09-27,2018-10-15,18 days
8,-8775,286,312,1259.73330,1125.00,1.182285e+06,0.106954,2018-10-26,2018-12-03,38 days
9,-8171,329,345,1497.55025,1324.00,1.418079e+06,0.115889,2018-12-26,2019-01-23,28 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 834 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                   74.595469
Equity Final [$]                10213999.5959
Equity Peak [$]                 11791760.3035
Return [%]                           2.139996
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                     0.43264
Volatility (Ann.) [%]               44.495565
Sharpe Ratio                         0.009723
Sortino Ratio                        0.015996
Calmar Ratio                         0.006477
Max. Drawdown [%]                  -66.795929
Avg. Drawdown [%]                   -18.45876
Max. Drawdown Duration     1449 days 00:00:00
Avg. Drawdown Duration      355 days 00:00:00
# Trades                                   66
Win Rate [%]                        53.030303
Best Trade [%]                       39.82143
Worst Trade [%]                    -18.616266
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6737,21,26,1484.31115,1393.65,-610784.16755,-0.061080,2017-10-20,2017-10-27,7 days
1,6745,27,32,1391.85575,1313.00,-531882.03375,-0.056655,2017-10-30,2017-11-06,7 days
2,6656,33,35,1330.55380,1255.90,-496895.69280,-0.056107,2017-11-07,2017-11-09,2 days
3,6340,36,46,1318.49440,1332.00,85625.50400,0.010243,2017-11-10,2017-11-24,14 days
4,-6343,55,95,1331.37690,1218.00,719149.67670,0.085158,2017-12-07,2018-02-01,56 days
...,...,...,...,...,...,...,...,...,...,...
61,-22640,1137,1155,408.96555,410.00,-23419.94800,-0.002529,2022-04-26,2022-05-26,30 days
62,-22365,1168,1173,412.94575,428.00,-336688.30125,-0.036456,2022-06-14,2022-06-21,7 days
63,-21602,1174,1186,411.95070,405.00,150149.02140,0.016873,2022-06-22,2022-07-08,16 days
64,-22073,1189,1224,409.96060,366.00,970342.32380,0.107231,2022-07-13,2022-09-02,51 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7590,31,32,1317.44620,1313.0,33746.65800,0.003375,2017-11-03,2017-11-06,3 days
1,-7616,33,34,1317.44620,1287.0,231878.25920,0.023110,2017-11-07,2017-11-08,1 days
2,-8136,35,36,1261.72340,1312.0,-409050.41760,-0.039848,2017-11-09,2017-11-10,1 days
3,-7550,37,38,1305.50560,1343.0,-283082.72000,-0.028720,2017-11-13,2017-11-14,1 days
4,-7110,39,40,1346.30265,1386.0,-282248.15850,-0.029486,2017-11-15,2017-11-16,1 days
...,...,...,...,...,...,...,...,...,...,...
406,-8861,1216,1217,345.28235,350.0,-41803.09665,-0.013663,2022-08-23,2022-08-24,1 days
407,-8640,1218,1219,349.26255,344.0,45468.43200,0.015068,2022-08-25,2022-08-26,1 days
408,-8481,1220,1221,361.20315,358.0,27165.91515,0.008868,2022-08-29,2022-08-30,1 days
409,-8580,1222,1223,360.20810,365.0,-41114.50200,-0.013303,2022-08-31,2022-09-01,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 811 configurations.
  output = _optimize_grid()


Start                     2017-09-21 00:00:00
End                       2022-09-20 00:00:00
Duration                   1825 days 00:00:00
Exposure Time [%]                    91.18123
Equity Final [$]                 500568.39605
Equity Peak [$]                 10378079.7568
Return [%]                         -94.994316
Buy & Hold Return [%]              -77.918935
Return (Ann.) [%]                   -45.69472
Volatility (Ann.) [%]               15.368389
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -95.270566
Avg. Drawdown [%]                  -32.759877
Max. Drawdown Duration     1764 days 00:00:00
Avg. Drawdown Duration      595 days 00:00:00
# Trades                                  513
Win Rate [%]                        39.766082
Best Trade [%]                      14.800104
Worst Trade [%]                    -13.766076
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-7567,30,31,1321.42640,1324.0,-19474.43120,-0.001948,2017-11-02,2017-11-03,1 days
1,-7639,32,33,1306.50065,1324.0,-133677.53465,-0.013394,2017-11-06,2017-11-07,1 days
2,-7689,34,35,1280.62935,1268.0,97107.07215,0.009862,2017-11-08,2017-11-09,1 days
3,-7616,36,37,1305.50560,1312.0,-49461.35040,-0.004975,2017-11-10,2017-11-13,3 days
4,-7404,38,39,1336.35215,1353.0,-123260.68140,-0.012458,2017-11-14,2017-11-15,1 days
...,...,...,...,...,...,...,...,...,...,...
508,-1605,1215,1216,345.28235,347.0,-2756.82825,-0.004975,2022-08-22,2022-08-23,1 days
509,-1583,1217,1218,348.26750,351.0,-4325.54750,-0.007846,2022-08-24,2022-08-25,1 days
510,-1598,1219,1220,342.29720,363.0,-33083.07440,-0.060482,2022-08-26,2022-08-29,3 days
511,-1443,1221,1222,356.22790,362.0,-8329.14030,-0.016203,2022-08-30,2022-08-31,1 days


<Strategy SMAandRSI_WithShortPosition(SMA_short=10,SMA_long=20,upper_bound=50,lower_bound=10,rsi_window=28)>

In [ ]:
df_returns_sorted = df_returns[["_strategy", "Start", "End", "Duration", "Exposure Time [%]", "Equity Final [$]", "Equity Peak [$]", "Return [%]", "Buy & Hold Return [%]", "Return (Ann.) [%]", "Volatility (Ann.) [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio", "Max. Drawdown [%]", "Avg. Drawdown [%]", "Max. Drawdown Duration", "Avg. Drawdown Duration", "# Trades", "Win Rate [%]", "Best Trade [%]", "Worst Trade [%]", "Avg. Trade [%]", "Max. Trade Duration", "Avg. Trade Duration", "Profit Factor", "Expectancy [%]", "SQN", 
]].sort_values(by = "Equity Final [$]", ascending = False)
df_returns_sorted.to_csv("CombinationStrategies.csv")